In [ ]:
import numpy as np
import pylab as plt

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


import sys
sys.path.append("..")
from fieldfinder import SpectralIndex #, write_NDVI_mask



[PlanetScope Ortho Scene Product (3B)](https://notebook.community/planetlabs/notebooks/jupyter-notebooks/toar/toar_planetscope)
MS radiance

Planet Labs' analytic data products are reported in units of radiance: $W*m^{-2}*sr^{-1}$

That means that every pixel in an analytic tiff has a natural language interpretation: "How much light was captured over this spot of ground?"

But over the course of a day or year, the number of photons that the sun shines on the scene rises and falls. If you naively compare the radiance values of two scenes over the same spot on Earth from the same satellite but a few weeks (or even just hours!) apart, you will likely find dramatic radiance differences even if nothing on the ground has changed!

In addition to this variation, each of Planet Labs' 150+ satllites have small amounts of variation in their spectral filters which yields slight differences in radiance measurements, even from two satellites taking pictures of the same exact place at the same exact moment!

Top of Atmosphere Reflectance is extremely useful because it is an apples-to-apples comparable number from any satellite over any location that does not change with time of day or time of year unless the content of the scene changes. It is very commonly used in GIS applications which compute spectral indices such as NDVI or NDWI.

$$ \begin{equation*} \mathbf{img}_{reflectance} = \begin{vmatrix} a \\ b \\ c \\ d \end{vmatrix} \times \mathbf{img}_{radiance} \end{equation*} $$
The four coefficients $a, b, c, d$ are calculated and provided with every analytic image that Planet provides and can be used as simple scaling factors to convert from radiance to reflectance. Their values change with the image's local time of day and time of year, and do so uniquely per satellite.

**Most spectral indices are defined in terms of reflectance, not radiance.**

- what is the actual data in the raw image?
- can we get the coefficients from anywhere?

From the PlanetScope definition page:
As a last step, the spatially and temporally adjusted datasets are transformed from digital number values into physical based radiance values (scaled to W * m²strμm)*100)

most definitions of NDVI seem to be based on reflectance values.  The data provided are radiance values.  Need the metadata to get the coefficients to turn into reflectance values.

from: https://eoscience.esa.int/landtraining2017/files/materials/D4P1b2I.pdf
> To calculate NDVI we have to convert radiance to reflectance

from: https://gis.stackexchange.com/questions/171453/is-the-reflectance-required-to-get-the-ndvi-for-landsat-8-images
> NDVI is defined for any two bands with near-infrared and infrared data (it is an empirical remote sensing index). As such, you can calculate it straight from the DNs. This is mostly OK if you are only classifying or analyzing vegetation on a single image without significant atmospheric effects (cirrus clouds...)

> However, if you are performing change detection (and therefore analysing two or more images), you will need to cope with two steps:

> Radiometric calibration converts the DNs to radiance using sensor-specific calibration equations. For Landsat, these can be found on the project website or built in most of the good software packages.

> Atmospheric correction tries to convert the radiance to reflectance (a dimensionless number describing the ratio of reflected radiation on the incoming radiation in the specific part of the spectrum). For this, atmosphere models are used such as QUAC or FLAASH.


from https://assets.planet.com/docs/Planet_Combined_Imagery_Product_Specs_letter_screen.pdf

What's in the provided data file?
File name format for PlanetScope Ortho Scene Product (3B)
YYYYMMDD_UTC-TIME_SAT-ID_PRODLVL_PRODTYPE

"<acquisition date>_<acquisition time>_<satellite_id>_<productLevel><bandProduct>.<extension>"

So this image was captured on August 27, 2021 at 10 am (Central Time).  The satellite ID is 60.  2262 might be a tile ID?  

3B is the product level.  This is the 'analytic ortho-scene product'. From the product description:

>Orthorectified, scaled Top of Atmosphere Radiance (at sensor) or Surface Reflectance image product suitable for analytic and visual applications. This product has scene based framing and projected to a cartographic projection.

Analytic MS is the product type. 8b stands for 8 band.

This is from a PlanetScope SuperDove satellite, which produces 8-band images.

Coastal Blue 431-452 nm
Blue: 465-515 nm
Green I: 513. - 549 nm
Green: 547. - 583 nm
Yellow: 600-620 nm
Red: 650 - 680 nm
Red-Edge: 697 - 713 nm
NIR: 845 - 885 nm


from https://developers.planet.com/docs/data/planetscope/:
Analytic (analytic) assets are orthorectified, calibrated, multispectral imagery products that have been corrected for sensor artifacts and terrain distortions, and transformed to Top of Atmosphere (at-sensor) radiance.

https://www.l3harrisgeospatial.com/Learn/Blogs/Blog-Details/ArtMID/10198/ArticleID/16278/Digital-Number-Radiance-and-Reflectance
Typically, for quantitative analysis of multispectral or hyperspectral image data, radiance images are corrected to reflectance images.

## Analysis with Rasterio

https://developers.planet.com/docs/planetschool/calculate-an-ndvi-in-python/
https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/vegetation-indices-in-python/calculate-NDVI-python/

Rabbit Hole!
https://yceo.yale.edu/how-convert-landsat-dns-top-atmosphere-toa-reflectance

[WGS 84 / UTM zone 15N](https://epsg.io/32615)

Output to EPSG 4326, or WGS 84 - Lat/Long coordinates

136 million pixels in each image.

NDVI is the normalized difference between the RED and NIR values
$$NDVI = \frac{NIR - R}{NIR + R}$$

### Create a Spectral Index (NDVI)

In [ ]:
filename = '../data/example_image.tif'

ndvi = SpectralIndex(filename)

plt.figure(figsize=(12,8))
plt.imshow(ndvi.values)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(ndvi.get_mask(0.65))
plt.show()

In [ ]:
out_proj = 'EPSG:4326'
output_file = '/Users/brendon/Projects/challenges/farmfinder/data/transformed_example_module.tif'

ndvi.write_mask(output_file, threshold=0.65, out_proj=out_proj)

Convenience function for creating a mask file with a single function.

In [ ]:
filename = '../data/example_image.tif'
threshold = 0.65
out_proj = 'EPSG:4326'
output_file = '/Users/brendon/Projects/challenges/farmfinder/data/transformed_example_single_single_from_class.tif'
SpectralIndex.create_mask_file(filename, output_file, threshold, out_proj)